In [2]:
import numpy as np

## Second order band-pass filter

### Continuous transfer function
The prototype low-pass filter transfer function with $\omega_0 = 1$ rad/s is
$$H(s) = \frac{1}{s + 1}$$
To derive the band-pass filter, use the band-pass transformation (refer to https://en.wikipedia.org/wiki/Prototype_filter):
$$s \rightarrow  Q\left(\frac{s}{\omega_0} + \frac{\omega_0}{s} \right)$$ with $$Q = \dfrac{\omega_0}{\Delta \omega}$$
where $\omega_0$ is the center of the band, and $\Delta \omega$ is the "width" of the band.

So
\begin{align*}
H(s) &= \frac{1}{ Q\left(\frac{s}{\omega_0} + \frac{\omega_0}{s} \right) + 1} \\
&= \frac{\omega_0 s}{ Q \left( s^2 + \omega_0^2 \right) + \omega_0 s} \\
&= \frac{(\omega_0/Q) s}{ s^2 + (\omega_0/Q) s + \omega_0^2} 
\end{align*}

### Discrete transfer function
Computing the discrete transfer function using Tustin's method (bilinear transform), set $s = \frac{2}{\Delta t} \left( \frac{1-z^{-1}}{1+z^{-1}} \right)$, so
\begin{align*} 
H(z)  &=\frac{(\omega_0/Q) \left( \frac{2}{\Delta t} \frac{1-z^{-1}}{1+z^{-1}} \right) }{ \left( \frac{2}{\Delta t} \frac{1-z^{-1}}{1+z^{-1}} \right)^2 + (\omega_0/Q)  \left( \frac{2}{\Delta t} \frac{1-z^{-1}}{1+z^{-1}} \right) + \omega_0^2} 
\\ &= \frac{ (2 \Delta t \omega_0 /Q) + (- 2 \Delta t \omega_0 /Q ) z^{-2} }{(\Delta t^2 \omega_0^2 + 2 \Delta t \omega_0/Q + 4) + ( 2\Delta t^2 \omega_0^2  - 8 )z^{-1} + (\Delta t^2 \omega_0^2 -2 \Delta t \omega_0/Q + 4)z^{-2} }
 \end{align*}
 
Set $\alpha = \omega_0 \Delta t $
\begin{align*} 
H(z) &= \frac{ (2 \alpha /Q) + (- 2 \alpha /Q ) z^{-2} }{(\alpha^2 + 2 \alpha/Q + 4) + ( 2\alpha^2  - 8 )z^{-1} + (\alpha^2 -2 \Delta t \omega_0/Q + 4)z^{-2} }
 \end{align*}

We want to find the filter coefficients for the discrete update:
$$y[n] = a_1 y[n-1] + a_2 y[n-2] + ... + b_0 x[n] + b_1 x[n-1] + ...$$

The coefficients can be taken directly from the discrete transfer function of the filter in the form:
$$H(z) = \frac{b_0 + b_1 z^{-1} + b_2 z^{-2} + \ldots}{1 - a_1 z^{-1} - a_2 z^{-2} + \ldots}$$

Set $D = \alpha^2 + 2 \alpha/Q + 4$, then
Setting $\alpha = \alpha^2 + 2 \alpha/Q + 4$, the transfer function is
\begin{align*} 
H(z) &= \frac{ (2 \alpha /(QD)) + (- 2 \alpha /(QD) ) z^{-2} }{1+ ( 2\alpha^2  - 8 )/D z^{-1} + (\alpha^2 -2 \alpha/Q + 4)/D z^{-2} }
 \end{align*}
 So the coefficients are



$$ b_0 = \frac{2 \alpha}{QD}\quad b_1 = 0 \quad  b_2 = - \frac{2 \alpha}{QD} \quad \text{and} \quad a_1 = - \frac{ 2\alpha^2  - 8 }{D} \quad a_2 = - \frac{\alpha^2 - 2\alpha/Q + 4 }{D} $$

### Arduino code
On the arduino code, we compute the coefficients as

<pre>
float alpha = omega0*dt;
Q = omega0/domega;
float D = pow(alpha,2) + 2*alpha/Q + 4;
b[0] = 2*alpha/(Q*D);
b[1] = -b[0];
a[0] = -(2*pow(alpha,2) - 8)/D;
a[1] = -(pow(alpha,2) - 2*alpha/Q + 4)/D;
</pre>

In [12]:
# Example coefficients for testing
dt = 1/1.0e3;
omega0 = 30*2*np.pi;
domega = 2.0*np.pi*20.0;          # Width of the band (rad/s)
alpha = omega0*dt;
Q = omega0/domega;
D = pow(alpha,2) + 2*alpha/Q + 4;
a = [0,0,0];
b = [0,0,0];
b[0] = 2*alpha/(Q*D);
b[2] = -b[0];
a[0] = 0;
a[1] = -(2*pow(alpha,2) - 8)/D;
a[2] = -(pow(alpha,2) - 2*alpha/Q + 4)/D;

print(b)
print(a)

[0.05862741732593571, 0, -0.05862741732593571]
[0, 1.8495921419055101, -0.8827451653481285]


<hr>